# Generate timeline how characters pop on screen

> C64 is the best


In [8]:
from enum import Enum
class UEnum(Enum):
    USPACE = 0
    UA = 1
    UB = 2
    UC = 3
    UD = 4
    UE = 5
    UF = 6
    UG = 7
    UH = 8
    UI = 9
    UJ = 10
    UK = 11
    UL = 12
    UM = 13
    UN = 14
    UO = 15
    UP = 16
    UQ = 17
    UR = 18
    US = 19
    UT = 20
    UU = 21
    UV = 22
    UW = 23
    UX = 24
    UY = 25
    UZ = 26
    UHH = 27
    U2 = 28
    UDOT = 29
    UQUESTION = 30
    UPLUS = 31

# C64 colors
BLACK = 0
WHITE = 1
RED = 2
GREEN = 5
BLUE = 6
YELLOW = 7
ORANGE = 8
DARK_GRAY = 11
GRAY = 12
LIGHT_GRAY = 15


SCREEN_WIDTH = 13
SCREEN_HEIGHT = 5
QUESTIONMARK_COUNT = 16
RAND_CHARS_COUNT = 256-SCREEN_WIDTH*SCREEN_HEIGHT-7-1+QUESTIONMARK_COUNT-1


In [9]:
class UpdateItem:
    def __init__(self, ch, x, y, color):
        self.ch = ch
        self.x = x
        self.y = y
        self.color = color

    # def __str__(self):
    #     return f"UpdateItem(ch={self.ch}, \t\tx={self.x}, y={self.y}, \tcolor={self.color})"
    
    def code(self, step) -> str:
        return f".eval update_timeline.set({step}, UpdateItem({str(self.ch).replace('UEnum.', '')}, {self.x}, {self.y}, screen_color(WHITE, {self.color})))"

In [10]:
# Allocate list of UpdateItem objects
# Each item is one step in a timeline
# First QUESTIONMARK_COUNT steps will always use UQUESTION as the character
# x is random integer from <0,SCREEN_WIDTH) and y is random integer from <0,SCREEN_HEIGHT)
# color is from [0, 11, 12, 15]
# Next steps will use random character from UEnum except of [USPACE, UHH, UDOT, UQUESTION]

# Step 1: Allocate list of UpdateItem objects
import random
update_items = []

# Step 2: First QUESTIONMARK_COUNT steps with UQUESTION
for _ in range(QUESTIONMARK_COUNT):
    while True:
        x = random.randint(0, SCREEN_WIDTH - 1)
        y = random.randint(0, SCREEN_HEIGHT - 1)
        if not (y == 2 and 3 <= x <= 9):
            break
    color = random.choice([0, 11, 12, 15])
    update_items.append(UpdateItem(UEnum.UQUESTION, x, y, color))

# Step 3: Next steps with random character from UEnum except [USPACE, UHH, UDOT, UQUESTION]
excluded_chars = {UEnum.USPACE, UEnum.UHH, UEnum.UDOT, UEnum.UQUESTION}
valid_chars = [char for char in UEnum if char not in excluded_chars]
occupied_positions = set()

for idx, _ in enumerate(range(QUESTIONMARK_COUNT, RAND_CHARS_COUNT)):
    char = random.choice(valid_chars)
    # Generate unique (x, y) coordinates
    while True:
        x = random.randint(0, SCREEN_WIDTH - 1)
        y = random.randint(0, SCREEN_HEIGHT - 1)
        if len(occupied_positions) <= (SCREEN_WIDTH*SCREEN_HEIGHT - 7-1):
            # screen still not yet full
            if (x, y) not in occupied_positions and not (y == 2 and 3 <= x <= 9):
                break
        else:
            # screen full, override existing chars
            if not (y == 2 and 3 <= x <= 9):
                break
    color = random.choice([0, 11, 12, 15])
    update_items.append(UpdateItem(char, x, y, color))
    occupied_positions.add((x, y))

# Step 4: places USPACE randomly to occupied_positions
sz = len(occupied_positions)
for _ in range(sz):
    # Take random item from occupied_positions
    occupied_item = random.choice(list(occupied_positions))    
    update_items.append(UpdateItem(UEnum.USPACE, occupied_item[0], occupied_item[1], BLACK))
    occupied_positions.remove(occupied_item)


In [11]:
for idx, i in enumerate(update_items):
    print(i.code(idx))

.eval update_timeline.set(0, UpdateItem(UQUESTION, 7, 0, screen_color(WHITE, 12)))
.eval update_timeline.set(1, UpdateItem(UQUESTION, 10, 3, screen_color(WHITE, 12)))
.eval update_timeline.set(2, UpdateItem(UQUESTION, 5, 3, screen_color(WHITE, 12)))
.eval update_timeline.set(3, UpdateItem(UQUESTION, 4, 1, screen_color(WHITE, 15)))
.eval update_timeline.set(4, UpdateItem(UQUESTION, 2, 2, screen_color(WHITE, 11)))
.eval update_timeline.set(5, UpdateItem(UQUESTION, 7, 1, screen_color(WHITE, 12)))
.eval update_timeline.set(6, UpdateItem(UQUESTION, 9, 3, screen_color(WHITE, 11)))
.eval update_timeline.set(7, UpdateItem(UQUESTION, 10, 1, screen_color(WHITE, 0)))
.eval update_timeline.set(8, UpdateItem(UQUESTION, 12, 2, screen_color(WHITE, 11)))
.eval update_timeline.set(9, UpdateItem(UQUESTION, 5, 0, screen_color(WHITE, 0)))
.eval update_timeline.set(10, UpdateItem(UQUESTION, 6, 0, screen_color(WHITE, 11)))
.eval update_timeline.set(11, UpdateItem(UQUESTION, 5, 0, screen_color(WHITE, 11)))
.